In [ ]:
%%capture
!pip install transformers
!pip install sentence_transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/data/clean_data.csv')
df['desc_len'] = df['imones_aprasymas'].str.split().str.len()
df = df[(df['desc_len']<101) & (df['desc_len']>2)]

In [ ]:
df.sample(1)

,imones_kodas,imones_pavadinimas,imones_aprasymas,desc_len,Nace2,EmployeesLast,gr_stat_letter,gr_stat_code,gr_4digits
6008,302573051,recorrente uab,melioracijos darbai gatvių tiesimas hidrotechn...,12,421100.0,10.0,F,F42,4211.0


In [ ]:
from sentence_transformers import SentenceTransformer
labse_model = SentenceTransformer('sentence-transformers/LaBSE')

from transformers import AutoModel, AutoTokenizer
model_name = "EMBEDDIA/litlat-bert"

bert_model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/606M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/358 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

In [ ]:
def bert_embeddings(x):
  inputs = tokenizer(x, padding=True, truncation=True, return_tensors="pt", max_length=512)
  outputs = bert_model(**inputs)
  embedings = outputs.pooler_output
  return embedings.detach().numpy()[0]

def labse_embeddings(x):
  embeddings = labse_model.encode(x)
  return embeddings

In [ ]:
bert_embeddings = [bert_embeddings(i) for i in df.imones_aprasymas.tolist()]
bert = pd.DataFrame(bert_embeddings)
bert.to_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/embeddings/bert_embeddings.csv', index=False)

In [ ]:
labse_embeddings = [labse_embeddings(i) for i in df.imones_aprasymas.tolist()]
labse = pd.DataFrame(labse_embeddings)
labse.to_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/embeddings/labse_embeddings.csv', index=False)

In [ ]:
from gensim.models import Word2Vec
import numpy as np
tokenized_docs = []

for text in df['imones_aprasymas']:
  tokenized_docs.append(text.split())

model = Word2Vec(sentences=tokenized_docs, vector_size=300, workers=1, seed=42)

def vectorize(docs, model):
  features = []
  for tokens in docs:
    zero_vector = np.zeros(model.vector_size)
    vectors = []
    for token in tokens:
      if token in model.wv:
        try:
          vectors.append(model.wv[token])
        except KeyError:
          continue
    if vectors:
      vectors = np.asarray(vectors)
      avg_vec = vectors.mean(axis=0)
      features.append(avg_vec)
    else:
      features.append(zero_vector)
  return features

In [ ]:
vectorized_docs = vectorize(tokenized_docs, model=model)
w2v_embeddings = pd.DataFrame(vectorized_docs)
w2v_embeddings.to_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/embeddings/w2v_embeddings.csv', index=False)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=.7, min_df=.1)
tfidf = vectorizer.fit_transform(list(df["imones_aprasymas"]))

In [ ]:
tfidf_embeddings = pd.DataFrame(tfidf)
tfidf_embeddings.to_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/embeddings/tfidf_embeddings.csv', index=False)

In [ ]:
tfidf

<28021x2 sparse matrix of type '<class 'numpy.float64'>'
	with 9686 stored elements in Compressed Sparse Row format>